# Slice vcf with Various Modules

- pybedtools
- htseq
- pyvcf

HTSeq
-----

- Slicing is quite complex and we dont get easy access to call details

pybedtools
----------

- quite flaky with access to large vcf.gz tabixed files
- just use it for interval algebra, if necessary

pycvf
-----

- works a treat
- issues with running fetch in Python 2 dues to some Cython bugginess. Tests only pass in Py3

In [74]:
from pyfaidx import Fasta
from pybedtools import BedTool
import vcf

In [3]:
ls -lh ../test/test-data/vcf/*vcf*

-rwxrwxrwx  1 johnmccallum  staff   506M  8 Apr  2015 ../test/test-data/vcf/20Monkey_chr22_bwt2.vcf.gz*
-rwxrwxrwx  1 johnmccallum  staff   796M 27 Feb 08:16 ../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz*
-rw-r--r--  1 johnmccallum  staff   5.4K 28 Feb 07:51 ../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz.tbi
-rwxrwxrwx  1 johnmccallum  staff   4.2M  8 Apr  2015 ../test/test-data/vcf/Hort16A_x_Russell.RefHort16A.filterD1Q20d6.vcf.gz*


In [4]:
!for FILE in $(ls ../test/test-data/vcf/*.gz*); do md5 $FILE; done

MD5 (../test/test-data/vcf/20Monkey_chr22_bwt2.vcf.gz) = f6475bca32701b2469b8916231b76c74
MD5 (../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz) = 912171805eb4b91d1e38a4d5d5a3929e
MD5 (../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz.tbi) = f8e742e04b8708a43b4f5fe04f4be1bf
MD5 (../test/test-data/vcf/Hort16A_x_Russell.RefHort16A.filterD1Q20d6.vcf.gz) = cd3cac9b08ed524954d929b5e352f48c
MD5 (../test/test-data/vcf/Kiwifruit_pseudomolecule.fa.gz) = 7f8dbe4675232f48b7fed274304ce2e0


In [5]:
!conda info -e

Using Anaconda Cloud api site https://api.anaconda.org
# conda environments:
#
HTS                      /Users/johnmccallum/miniconda3/envs/HTS
_build                   /Users/johnmccallum/miniconda3/envs/_build
py2PCR                   /Users/johnmccallum/miniconda3/envs/py2PCR
py3-r-env                /Users/johnmccallum/miniconda3/envs/py3-r-env
py3markers            *  /Users/johnmccallum/miniconda3/envs/py3markers
root                     /Users/johnmccallum/miniconda3



### Load up an indexed reference and vcf

In [6]:
myfa= Fasta("../test/test-data/Kiwifruit_pseudomolecule.fa")
myvcf=vcf.Reader(filename='../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz')

In [10]:
!bgzip -dc  ../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz | awk '!/##/' | head

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	A-chinensis_Female_	AcMale_CK15_	PFR_001_	PFR_002_	PFR_003_	PFR_004_	PFR_005_	PFR_006_	PFR_007_	PFR_008_	PFR_009	PFR_010	PFR_011	PFR_012	PFR_013	PFR_014	PFR_015	PFR_016	PFR_019	PFR_021
9	1	.	A	T	21.9677	.	AB=0;ABP=0;AC=4;AF=0.111111;AN=36;AO=4;CIGAR=1X;DP=49;DPB=49;DPRA=0.711111;EPP=5.18177;EPPR=31.4368;GTI=2;LEN=1;MEANALT=1;MQM=31;MQMR=30.9318;NS=18;NUMALT=1;ODDS=4.00257;PAIRED=0.75;PAIREDR=0.772727;PAO=0;PQA=0;PQR=0;PRO=0;QA=126;QR=1524;RO=44;RPP=11.6962;RPPR=98.5551;RUN=1;SAF=3;SAP=5.18177;SAR=1;SRF=34;SRP=31.4368;SRR=10;TYPE=snp;technology.Illumina=1	GT:GQ:DP:RO:QR:AO:QA:GL	0/0:24.0547:3:3:119:0:0:0,-0.90309,-10	0/0:30.0753:6:5:195:0:0:0,-1.50515,-10	0/0:30.0753:5:5:172:0:0:0,-1.50515,-10	0/0:21.0444:2:2:68:0:0:0,-0.60206,-6.46	0/0:18.0341:1:1:35:0:0:0,-0.30103,-3.5	0/0:24.0547:3:3:107:0:0:0,-0.90309,-9.98667	1/1:0.000823243:2:0:0:2:64:-6.08,-0.60206,0	0/0:33.0856:6:6:232:0:0:0,-1.80618,-10	.	0/0:18.0341:1:1:41:0:0:0,-0.30103,-4.1	0/0:2

Can access a slice using [fetch](http://pyvcf.readthedocs.org/en/latest/API.html#vcf-reader)

Pythonic intervals are start/end, POS is 1-based vcf index

In [17]:
 for X in myvcf.fetch("9",10000,10100):
        print(X.POS,X.start,X.end,X.var_type,X.alleles)

10002 10001 10002 snp ['A', T]
10007 10006 10007 snp ['C', A]
10013 10012 10013 snp ['T', C]
10014 10013 10014 snp ['G', T]
10018 10017 10018 snp ['G', T]
10020 10019 10020 snp ['T', G, A]
10026 10025 10026 snp ['A', T, G]
10031 10030 10034 indel ['TACC', GACC, AACC, GACT]
10036 10035 10036 snp ['C', T]
10038 10037 10038 snp ['C', A]
10039 10038 10043 indel ['CTAAT', CTAAG, CTAAA, TTAAG]
10048 10047 10048 snp ['G', T]
10049 10048 10049 snp ['A', T]
10053 10052 10053 snp ['T', A]
10054 10053 10054 snp ['T', A]
10058 10057 10058 snp ['A', T]
10061 10060 10070 indel ['GATTCTCCAT', GATTCTCCAA, GATTCTCCAG, AATTTTCTAA]
10079 10078 10079 snp ['T', A]
10080 10079 10080 snp ['G', A]
10082 10081 10082 snp ['C', T]
10087 10086 10087 snp ['T', A]
10088 10087 10088 snp ['C', G]
10089 10088 10089 snp ['T', A]
10100 10099 10100 snp ['T', G, A]


To get length can just use end - start  

Fetch arg has our chromosome/contig ID

In [40]:
for  X in myvcf.fetch("9",10000,11000):
    if (X.var_type=='indel'):
        print(X.POS,X.var_subtype,X.start,X.end,X.affected_end,X.REF,X.ALT,X.alleles)

10031 unknown 10030 10034 10034 TACC [GACC, AACC, GACT] ['TACC', GACC, AACC, GACT]
10039 unknown 10038 10043 10043 CTAAT [CTAAG, CTAAA, TTAAG] ['CTAAT', CTAAG, CTAAA, TTAAG]
10061 unknown 10060 10070 10070 GATTCTCCAT [GATTCTCCAA, GATTCTCCAG, AATTTTCTAA] ['GATTCTCCAT', GATTCTCCAA, GATTCTCCAG, AATTTTCTAA]
10168 unknown 10167 10174 10174 GTACGGT [GTACGGG, GTACGGC, GCACGGG, ATATGGG, GTATGGG] ['GTACGGT', GTACGGG, GTACGGC, GCACGGG, ATATGGG, GTATGGG]
10250 unknown 10249 10253 10253 TCCC [CCCC, CCCT] ['TCCC', CCCC, CCCT]
10280 ins 10279 10288 10288 GAAATGCTA [GAAATGCTAAATGCTA] ['GAAATGCTA', GAAATGCTAAATGCTA]
10296 unknown 10295 10301 10301 GTTAGA [ATTAGA, ATTA] ['GTTAGA', ATTAGA, ATTA]
10305 del 10304 10307 10307 TAA [TA] ['TAA', TA]
10313 unknown 10312 10317 10317 GGTGG [CGTG, CGTGG] ['GGTGG', CGTG, CGTGG]
10318 unknown 10317 10327 10327 GTATATATAT [GTATATATATGT, GTGTGTGTGT, GTATATATGT] ['GTATATATAT', GTATATATATGT, GTGTGTGTGT, GTATATATGT]
10332 unknown 10331 10334 10334 GTG [GTGTTTATATG, GTT]

Probaby simplest to return a dict from comprehension, so we can the exclude our target.

**However** this would be less flexible than making a bed tool

would need to 

- pass in a target in Python Primer 3 target format

i.e. 0-based start  length

- exclude using 

In [75]:
myintdict={X.start:X.end-X.start for X  in myvcf.fetch("9",10200,10400)}

### Create  a BedTool from Vcf interval

In [107]:
["Chr9" + " " + str(X.start)+ " " +  str(X.end) for X  in myvcf.fetch("9",10200,10300)]

['Chr9 10201 10202',
 'Chr9 10202 10203',
 'Chr9 10206 10207',
 'Chr9 10210 10211',
 'Chr9 10213 10214',
 'Chr9 10215 10216',
 'Chr9 10218 10219',
 'Chr9 10225 10226',
 'Chr9 10226 10227',
 'Chr9 10227 10228',
 'Chr9 10232 10233',
 'Chr9 10233 10234',
 'Chr9 10241 10242',
 'Chr9 10243 10244',
 'Chr9 10247 10248',
 'Chr9 10249 10253',
 'Chr9 10255 10256',
 'Chr9 10257 10258',
 'Chr9 10260 10261',
 'Chr9 10261 10262',
 'Chr9 10262 10263',
 'Chr9 10271 10272',
 'Chr9 10279 10288',
 'Chr9 10295 10301']

need to join with newline into a string

In [110]:
testint=["Chr9" + " " + str(X.start)+ " " +  str(X.end) for X  in myvcf.fetch("9",10200,10300)]
"\n".join(testint)

'Chr9 10201 10202\nChr9 10202 10203\nChr9 10206 10207\nChr9 10210 10211\nChr9 10213 10214\nChr9 10215 10216\nChr9 10218 10219\nChr9 10225 10226\nChr9 10226 10227\nChr9 10227 10228\nChr9 10232 10233\nChr9 10233 10234\nChr9 10241 10242\nChr9 10243 10244\nChr9 10247 10248\nChr9 10249 10253\nChr9 10255 10256\nChr9 10257 10258\nChr9 10260 10261\nChr9 10261 10262\nChr9 10262 10263\nChr9 10271 10272\nChr9 10279 10288\nChr9 10295 10301'

In [112]:
testbed=BedTool("\n".join(testint),from_string=True)

In [113]:
print(testbed)

Chr9	10201	10202
Chr9	10202	10203
Chr9	10206	10207
Chr9	10210	10211
Chr9	10213	10214
Chr9	10215	10216
Chr9	10218	10219
Chr9	10225	10226
Chr9	10226	10227
Chr9	10227	10228
Chr9	10232	10233
Chr9	10233	10234
Chr9	10241	10242
Chr9	10243	10244
Chr9	10247	10248
Chr9	10249	10253
Chr9	10255	10256
Chr9	10257	10258
Chr9	10260	10261
Chr9	10261	10262
Chr9	10262	10263
Chr9	10271	10272
Chr9	10279	10288
Chr9	10295	10301



## Write a class

In [138]:
class VcfPrimerDesign:
    """A primer design object that is primed
    with genome reference and vcf variant data
    """
    def __init__(self,reference,vcf_file,desc):
        """
        Usage:  PrimerDesign(reference, vcf.gz, description)
        Initialise a design object with a  reference assembly and
        variant file(s)
        """
        self.reference = Fasta(reference)
        self.vcf=vcf.Reader(filename=vcf_file)
        self.desc=desc
        self.genome=self.reference.filename.replace("fa","fa.fai")
    
    
    def getseqslicedict(self,target,max_size):
        """Pass a bed target to a designer and get a dictionary
        slice that we can pass to P3
        """
        target_int=target.slop(b=max_size,g=self.genome)
        target_chrom=target[0].chrom
        target_start=target_int[0].start
        target_end=target_int[0].end
        offset=target_int[0].start
        sldic=dict(SEQUENCE_ID=self.desc)
        sldic['TARGET_ID']=target_chrom + "_" + str(target_start) +"_" + str(target_end)
        sldic['SEQUENCE_TEMPLATE']=str(self.reference[target_chrom][target_start:target_end].seq)
        #slice_annot=[(X.start -offset,X.length) for X in (self.annotations - target) if (X.chrom==target[0].chrom) & \
         #           (X.start > target_int[0].start) & (X.end < target_int[0].end)]
        slice_vars=[target_chrom + " " + str(X.start)+ " " +str(X.end) for X in self.vcf.fetch(target_chrom,target_start,target_end)]
        slice_annot=BedTool("\n".join(slice_vars),from_string=True)
        slice_annot=slice_annot-target
        sldic['SEQUENCE_EXCLUDED_REGION']=[(X.start,X.length) for X in slice_annot]
        sldic['SEQUENCE_TARGET']= (target[0].start -offset,target[0].length)
        return sldic



-----------------

Try Out 

In [139]:
mybdesigner=VcfPrimerDesign('../test/test-data/Kiwifruit_pseudomolecule.fa',
                            '../test/test-data/vcf/20Monkey_chr9_bwa.vcf.gz','Myb')

In [141]:
my_target=BedTool('9 10000 10001',from_string=True)
mybdesigner.getseqslicedict(my_target,200)

{'SEQUENCE_EXCLUDED_REGION': [(9800, 1),
  (9801, 1),
  (9804, 1),
  (9805, 1),
  (9806, 1),
  (9807, 1),
  (9808, 1),
  (9813, 1),
  (9814, 1),
  (9815, 1),
  (9817, 1),
  (9818, 1),
  (9820, 1),
  (9821, 1),
  (9828, 1),
  (9836, 4),
  (9842, 1),
  (9846, 1),
  (9848, 1),
  (9851, 1),
  (9854, 6),
  (9867, 1),
  (9870, 1),
  (9874, 3),
  (9877, 1),
  (9878, 1),
  (9879, 1),
  (9880, 1),
  (9881, 1),
  (9882, 1),
  (9884, 1),
  (9885, 1),
  (9887, 1),
  (9888, 2),
  (9890, 1),
  (9891, 1),
  (9893, 1),
  (9895, 1),
  (9896, 1),
  (9897, 1),
  (9898, 1),
  (9903, 1),
  (9904, 1),
  (9905, 1),
  (9906, 1),
  (9908, 1),
  (9911, 1),
  (9912, 4),
  (9916, 1),
  (9917, 1),
  (9918, 1),
  (9920, 1),
  (9921, 1),
  (9922, 1),
  (9923, 1),
  (9924, 1),
  (9925, 1),
  (9926, 1),
  (9928, 1),
  (9929, 1),
  (9930, 1),
  (9931, 1),
  (9933, 1),
  (9935, 1),
  (9938, 2),
  (9942, 1),
  (9944, 1),
  (9945, 1),
  (9946, 1),
  (9950, 1),
  (9951, 1),
  (9952, 1),
  (9955, 1),
  (9964, 1),
  (9967, 1

### Test with some cleaner vcf

In [1]:
from pcr_marker_design import design as d
from pybedtools import BedTool
import vcf

In [2]:
mybdesign=d.VcfPrimerDesign('../test/test-data/Kiwifruit_pseudomolecule_Chr.fa',
                                '../test/test-data/Chr9_Myb210.vcf.gz','MybTest')

In [3]:
mytarget=BedTool("Chr9 1390065 1390066",from_string=True)
mybdesign.getseqslicedict(mytarget,100)

{'SEQUENCE_EXCLUDED_REGION': [(82, 4),
  (92, 1),
  (113, 1),
  (126, 1),
  (127, 1),
  (139, 3),
  (150, 1),
  (164, 2),
  (192, 1)],
 'SEQUENCE_ID': 'MybTest',
 'SEQUENCE_TARGET': (100, 1),
 'SEQUENCE_TEMPLATE': 'TAAATTTATAAAAAATATATAAATATATATATATATAAATATATAAATATATATATATATAAATATATAAATATATATATATATATATATATATATATATATGTGAGAAAGTATTGATTAACCATAGCGGAGTAAACAACAGAGTTAGAAAAACTCTTTAAAAAAATGTGCTCTCTTTTGTGTGGGGAAAAAGCGCCACAG',
 'TARGET_ID': 'Chr9_1389965_1390166'}

In [4]:
test_seq="../test/test-data/Kiwifruit_pseudomolecule_Chr.fa"
vcffile= "../test/test-data/Chr9_Myb210.vcf.gz"
designer = d.VcfPrimerDesign(test_seq,vcffile,"MybTest")

In [5]:
target=BedTool("Chr9 1390065 1390066",from_string=True)

In [8]:
designer.getseqslicedict(target,100)

{'SEQUENCE_EXCLUDED_REGION': [(82, 4),
  (92, 1),
  (113, 1),
  (126, 1),
  (127, 1),
  (139, 3),
  (150, 1),
  (164, 2),
  (192, 1)],
 'SEQUENCE_ID': 'MybTest',
 'SEQUENCE_TARGET': (100, 1),
 'SEQUENCE_TEMPLATE': 'TAAATTTATAAAAAATATATAAATATATATATATATAAATATATAAATATATATATATATAAATATATAAATATATATATATATATATATATATATATATATGTGAGAAAGTATTGATTAACCATAGCGGAGTAAACAACAGAGTTAGAAAAACTCTTTAAAAAAATGTGCTCTCTTTTGTGTGGGGAAAAAGCGCCACAG',
 'TARGET_ID': 'Chr9_1389965_1390166'}

In [7]:
import pysam
pysam.__version__

'0.8.4'

**This ony passes in Python3**

-----------

Try to Design
----------

In [11]:
from pcr_marker_design import run_p3 as P3

In [ ]:
p3_test_globals={
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_INTERNAL_OLIGO': 1,
        'PRIMER_INTERNAL_MAX_SELF_END': 8,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 20.0,
        'PRIMER_MAX_GC': 80.0,
        'PRIMER_MAX_POLY_X': 100,
        'PRIMER_INTERNAL_MAX_POLY_X': 100,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 12,
        'PRIMER_MAX_SELF_END': 8,
        'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        'PRIMER_PAIR_MAX_COMPL_END': 8,
        'PRIMER_PRODUCT_SIZE_RANGE': [[75,100],[100,125],[125,150],[150,175],[175,200],[200,225]],
    }